In [1]:
%matplotlib inline
from numpy import log,sin,sinh,sqrt
import numpy as np
import emcee
import matplotlib.pyplot as plt
import corner
#模型公式
def model(z,theta):
    #z代表红移，dL代表光度距离，mu为距离模数，theta代表模型中的参数
    mu=5.*np.log10(dL(z,theta))
    return mu
#计算光度距离
def dL(z,theta):
    omegam,omegak, omega_lambda = theta
    #omega取值不同，计算dL的公式不同
    if(omegak<0):
        f=(1+z)*sin(integral(z,theta)*sqrt(-omegak))/sqrt(-omegak)
    if(omegak==0):
        f=(1+z)*integral(z,theta)
    if(omegak>0):
        f=(1+z)*sinh(integral(z,theta)*sqrt(omegak))/sqrt(omegak)
    return f


#辛普森法计算模型公式里面的积分
def integral(z,theta):
    #根据之前作业里面的经验，50步的相对误差为10^-7左右
    N=50
    #a,b为积分区间，N为分割的份数
    a=0.
    b=z
    h=(b-a)/N
    sum_1=function(a,theta)+function(b,theta)   #首尾两项的和
    sum_odd=0.                       #奇数项的和
    for k in range(1,N,2):
        sum_odd=sum_odd+function(a+k*h,theta)
    sum_even=0.                     #偶数项的和
    for k in range(2,N,2):
        sum_even=sum_even+function(a+k*h,theta)
    sum_total=sum_1+4*sum_odd+2*sum_even 
    return sum_total*h/3            #返回积分总和


def function(z,theta):
    omegam, omegak, omega_lambda = theta
    #哈勃常数
    H0=67.8 
    #omegar=1-omega_lambda-omegam-omegak ，考虑约束条件
    omegar=1e-4 #不考虑约束条件，将其设定为基准值
    EZ=H0*np.sqrt(abs((1+z)**3*omegam+(1+z)**4*omegar+(1+z)**2*omegak+omega_lambda))
    return 1./EZ

    
def lnlike(theta, x, y, inv_sigma2):
    omegam, omegak,omega_lambda = theta
    #model_com为MCMC链的每一步的参数计算的距离模数结果,代表论文里面的μth
    model_com = model(x,theta)
    #y为代入参数的基准值计算的距离模数结果，代表μobs
    d=-0.5*(np.sum((y-model_com)**2*inv_sigma2 - np.log(inv_sigma2)))
    return d


def lnprior(theta):
    omegam,omegak,omega_lambda = theta
    if -20. < omegam < 10 and -5< omega_lambda < 5.0 and -10<omegak<10:
        return 0.0
    return -np.inf

def lnprob(theta, x, y, yerr):
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf
   
    return lp + lnlike(theta, x, y, yerr)


#导入红移文件
datum = np.loadtxt(r'C:\Users\zhang\Desktop\redshift.dat')
x     = datum[:]
#三个宇宙学参数，omegam,omegak,omega_lambda对应的基准值
fid=0.3,0.,0.7
#代入参数基准值算出来的距离模数值
y    =model(x,fid) 
N = len(x)

#误差项，具体每一项含义请参见论文
sigma_int = 0.09
sigma_meas = 0.08
sigma_lens = 0.07
peculiar_v = 400.
#最后一项误差项和红移相关
inv_sigma2= 1./( \
       sigma_int**2 \
       + (sigma_meas)**2 \
       + (sigma_lens*x)**2 \
       +((5/sqrt(2.)/log(10.)*peculiar_v/3.e5)/x)**2 )

ndim, nwalkers = 3, 10


#马科夫链的起点放在了最佳拟合值附近，在小范围内搜索最佳拟合值
pos = [[0.3,0, 0.7] + 1e-1*np.random.randn(ndim) for i in range(nwalkers)]
#试过放在比较远的地方，进行大范围搜索，基本得不到正常的对角图结果
import emcee
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=(x, y, inv_sigma2))

sampler.run_mcmc(pos, 2000)
samples = sampler.chain[:, 500:, :].reshape((-1, ndim))
fig = corner.corner(samples, labels=["$\Omega_m$",'$\Omega_k$', "$\Omega_{\Lambda}$"],
                      truths=[0.3,0, 0.7])
omega_m, omega_k,omega__lambda = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                             zip(*np.percentile(samples, [16, 50, 84],
                                                axis=0)))
print("omega_m:", omega_m)
print("omega_k:" ,omega_k)
print("omega_Lambda: ",omega__lambda)
plt.savefig('mcmc_result5.png',dpi=300)
plt.show()



OSError: C:\Users\zhang\Desktop\redshift.dat not found.